## Imports

In [1]:
import sys
import os
import argparse
import time
import random
import math
import numpy as np
from scipy.special import logsumexp

import torch
import torchvision
from PIL import Image
import torch.nn as nn
import torch.optim as optim
import torchvision.transforms as tfs

from tensorboardX import SummaryWriter


from utils import kNN, AverageMeter, py_softmax

## Training parameters

In [2]:
#data
datadir = "/root/data/Multivariate_ts"

# optimization
lamb = 0.1      # SK lambda-parameter
nopts = 400    # number of SK-optimizations
epochs = 400   # numbers of epochs
momentum = 0.9 # sgd momentum
exp = './resnet1d_exp' # experiments results dir


# other
devc='0'  # cuda device
batch_size = 100
lr=0.03     #learning rate
alr=0.03    #starting learning rate

knn_dim = 20
best_acc = 0  # best test accuracy
start_epoch = 0  # start from epoch 0 or last checkpoint epoch

In [3]:
device = torch.device('cuda:' + devc) if torch.cuda.is_available() else torch.device('cpu')
print(f"GPU device: {torch.cuda.current_device()}")

GPU device: 0


## Model parameters (AlexNet in that case)

In [4]:
hc=10       # number of heads
ncl=128       # number of clusters

numc = [ncl] * hc
# # (number of filters, kernel size, stride, pad) for AlexNet, two vesions
# CFG = {
#     'big': [(96, 11, 4, 2), 'M', (256, 5, 1, 2), 'M', (384, 3, 1, 1), (384, 3, 1, 1), (256, 3, 1, 1), 'M'],
#     'small': [(64, 11, 4, 2), 'M', (192, 5, 1, 2), 'M', (384, 3, 1, 1), (256, 3, 1, 1), (256, 3, 1, 1), 'M']
# }

## Data Preparation

In [5]:
import pandas as pd
import numpy as np
from scipy.io import arff
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn import preprocessing
from sktime.utils.load_data import load_from_tsfile_to_dataframe

from tqdm import tqdm
import os
from datetime import datetime

In [6]:
def preproc(X, dims_num):
    X = X.applymap(np.array)
    cols_lst = X.columns
    
    for col_name in cols_lst:
        dimension_values = np.dstack(list(X[col_name].values))[0].T
        time_steps_num = dimension_values.shape[1]
        
        for i in range(time_steps_num):
            one_step_values = dimension_values[:, i]
            X[col_name + f'_{i}'] = one_step_values
    
    X = X.drop(columns=cols_lst)
    
    step = int(X.shape[1] / dims_num)
    X_3d = []
    init = 0
    for dim_num in range(dims_num):
        X_3d.append(X.iloc[:, init:init + step])
        init += step
    X_3d = np.dstack(X_3d)
    return X_3d

def load_file(filepath, dims_num, is_arff=False):
    if is_arff:
        data = arff.loadarff(filepath) 
        data = pd.DataFrame(data[0])
        X = data.iloc[:, :-1] # [30, 65] x 4 times
        y = data.iloc[:, -1]
        return X.values, y.values
    else:
        X, y = load_from_tsfile_to_dataframe(filepath)
        X = preproc(X, dims_num)
        return X, y

def load_group(prefix, filenames, dims_num, is_arff=False):
    loaded = []
    if is_arff:
        for name in filenames: 
            X, y = load_file(prefix + "/" + name, dims_num, is_arff=is_arff) # [30, 65]
            loaded.append(X)
            # stack group so that features are the 3rd dimension 
        loaded = np.dstack(loaded) # [30, 65, 4]
    else:
        loaded, y = load_file(prefix + "/" + filenames[0], dims_num, is_arff=is_arff) # [30, 65, 4]
    return loaded, y

def load_dataset_group(folder_path, ds_path, dims_num, is_train=True, label_enc=False, is_arff=False): 
    filenames = []
    extension = ".arff" if is_arff else ".ts"
    postfix = "_TRAIN" if is_train else "_TEST"
    
    if is_arff:
        for dim_num in range(1, dims_num + 1):
            filenames.append(ds_path + str(dim_num) + postfix + extension)
    else:
        filenames.append(ds_path + postfix + extension)

    X, y = load_group(folder_path, filenames, dims_num, is_arff=is_arff) # [30, 65, 4]
    X = torch.from_numpy(np.array(X, dtype=np.float64))
    if label_enc:
        le = preprocessing.LabelEncoder()
        y = le.fit_transform(y)
        y = torch.from_numpy(np.array(y, dtype=np.int32))
    else:
        y = torch.from_numpy(np.array(y, dtype=np.int32)) - 1
    X = X.transpose(1, 2)
    return X, y

def load_dataset(folder_path, ds_path, dims_num, label_enc=False, is_arff=False): 
    X_train, y_train = load_dataset_group(folder_path, ds_path, dims_num, 
                                          is_train=True, label_enc=label_enc, is_arff=is_arff) 
    X_test, y_test = load_dataset_group(folder_path, ds_path, dims_num, 
                                        is_train=False, label_enc=label_enc, is_arff=is_arff) 
    return X_train, y_train, X_test, y_test

def iterate_minibatches(inputs, targets, batchsize, shuffle=False):
    assert len(inputs) == len(targets)
    if shuffle:
        indices = np.random.permutation(len(inputs))
    for start_idx in range(0, len(inputs) - batchsize + 1, batchsize):
        if shuffle:
            excerpt = indices[start_idx:start_idx + batchsize]
        else:
            excerpt = slice(start_idx, start_idx + batchsize)
        yield inputs[excerpt], targets[excerpt], excerpt

In [7]:
#ds_path = "ERing/ERingDimension"
#ds_path = "SpokenArabicDigits/SpokenArabicDigitsDimension"
# dims_num = 13
# num_classes = 10
# magic_dim = 4500
label_enc = True
ds_path = "LSST/LSST"
dims_num = 6
num_classes = 14
magic_dim = 2304

# ds_path = "HandMovementDirection/HandMovementDirection"
# label_enc = True
# dims_num = 10
# num_classes = 4
# magic_dim = 25600
# ds_path = "PenDigits/PenDigits"
# label_enc = True
# dims_num = 2
# num_classes = 4
# magic_dim = 512

X_train, y_train, X_test, y_test = load_dataset(datadir, ds_path, dims_num, label_enc=False, is_arff=False)
print("X_train.shape:", X_train.shape, "\ny_train.shape:", y_train.shape)
print("X_test.shape:", X_test.shape, "\ny_test.shape:", y_test.shape)

X_train.shape: torch.Size([2459, 6, 36]) 
y_train.shape: torch.Size([2459])
X_test.shape: torch.Size([2466, 6, 36]) 
y_test.shape: torch.Size([2466])


In [8]:
N = X_train.shape[0]
N

2459

In [9]:
X_train.shape

torch.Size([2459, 6, 36])

## Model, ResNet

In [10]:
import torch.nn as nn
import math

__all__ = ['resnetv1','resnetv1_18']

def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv1d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class Normalize(nn.Module):
    def __init__(self, power=2):
        super(Normalize, self).__init__()
        self.power = power

    def forward(self, x):
        norm = x.pow(self.power).sum(1, keepdim=True).pow(1. / self.power)
        out = x.div(norm)
        return out

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm1d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm1d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv1d(inplanes, planes, kernel_size=1, bias=False)
        self.bn1 = nn.BatchNorm1d(planes)
        self.conv2 = nn.Conv1d(planes, planes, kernel_size=3, stride=stride,
                               padding=1, bias=False)
        self.bn2 = nn.BatchNorm1d(planes)
        self.conv3 = nn.Conv1d(planes, planes * 4, kernel_size=1, bias=False)
        self.bn3 = nn.BatchNorm1d(planes * 4)
        self.relu = nn.ReLU(inplace=True)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)

        out = self.conv3(out)
        out = self.bn3(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out

class ResNet(nn.Module):
    def __init__(self, block, layers, in_channel=3, width=1, num_classes=[1000]):
        self.inplanes = 16
        super(ResNet, self).__init__()
        self.headcount = len(num_classes)
        self.base = int(16 * width)
        self.features = nn.Sequential(*[                                                     # [100, 8, 18]
                            nn.Conv1d(in_channel, 16, kernel_size=3, padding=1, bias=False), # [100, 16, 36]
                            nn.BatchNorm1d(16),
                            nn.ReLU(inplace=True),
                            self._make_layer(block, self.base, layers[0]),                   # [100, 16, 36]
                            self._make_layer(block, self.base * 2, layers[1]),               # [100, 32, 36]
                            self._make_layer(block, self.base * 4, layers[2]),               # [100, 64, 36]
                            self._make_layer(block, self.base * 8, layers[3]),               # [100, 128, 36]
                            nn.AvgPool1d(2),                                                 # [100, 128, 18]
        ])
    
        if len(num_classes) == 1:
            self.top_layer = nn.Sequential(nn.Linear(magic_dim, num_classes[0]))
        else:
            for a, i in enumerate(num_classes):
                setattr(self, "top_layer%d" % a, nn.Linear(magic_dim, i))
            self.top_layer = None
        for m in self.features.modules():
            if isinstance(m, nn.Conv1d):
                n = m.kernel_size[0] * m.out_channels
                m.weight.data.normal_(0, math.sqrt(2. / n))
            elif isinstance(m, nn.BatchNorm1d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv1d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm1d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.features(x.float())
        out = out.view(out.size(0), -1)
        if self.headcount == 1:
            if self.top_layer:
                out = self.top_layer(out)
            return out
        else:
            outp = []
            for i in range(self.headcount):
                outp.append(getattr(self, "top_layer%d" % i)(out))
            return outp

def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    return model

def resnetv1_18(num_classes=[1000]):
    """Encoder for instance discrimination and MoCo"""
    return resnet18(num_classes=num_classes)

## Sinkhorn-Knopp optimization

In [11]:
def optimize_L_sk(PS):
    N, K = PS.shape
    tt = time.time()
    PS = PS.T  # now it is K x N
    r = np.ones((K, 1)) / K
    c = np.ones((N, 1)) / N
    PS **= lamb  # K x N
    inv_K = 1. / K
    inv_N = 1. / N
    err = 1e3
    _counter = 0
    while err > 1e-2:
        r = inv_K / (PS @ c)  # (KxN)@(N,1) = K x 1
        c_new = inv_N / (r.T @ PS).T  # ((1,K)@(KxN)).t() = N x 1
        if _counter % 10 == 0:
            err = np.nansum(np.abs(c / c_new - 1))
        c = c_new
        _counter += 1
        
    print("error: ", err, 'step ', _counter, flush=True)  # " nonneg: ", sum(I), flush=True)
    # inplace calculations.
    PS *= np.squeeze(c)
    PS = PS.T
    PS *= np.squeeze(r)
    PS = PS.T
    argmaxes = np.nanargmax(PS, 0)  # size N
    newL = torch.LongTensor(argmaxes)
    selflabels = newL.to(device)
    PS = PS.T
    PS /= np.squeeze(r)
    PS = PS.T
    PS /= np.squeeze(c)
    sol = PS[argmaxes, np.arange(N)]
    np.log(sol, sol)
    cost = -(1. / lamb) * np.nansum(sol) / N
    print('cost: ', cost, flush=True)
    print('opt took {0:.2f}min, {1:4d}iters'.format(((time.time() - tt) / 60.), _counter), flush=True)
    return cost, selflabels

def opt_sk(model, selflabels_in, epoch):
    if hc == 1:
        PS = np.zeros((N, ncl))
    else:
        PS_pre = np.zeros((N, magic_dim)) # knn_dim
    
    for batch_idx, (data, _, _selected) in enumerate(iterate_minibatches(X_train, y_train, batch_size, shuffle=True)):
        data = data.to(device)#cuda()
        if hc == 1:
            p = nn.functional.softmax(model(data), 1)
            PS[_selected, :] = p.detach().cpu().numpy()
        else:
            p = model(data.float())
            PS_pre[_selected, :] = p.detach().cpu().numpy()
    if hc == 1:
        cost, selflabels = optimize_L_sk(PS)
        _costs = [cost]
    else:
        _nmis = np.zeros(hc)
        _costs = np.zeros(hc)
        nh = epoch % hc  # np.random.randint(args.hc)
        print("computing head %s " % nh, end="\r", flush=True)
        tl = getattr(model, "top_layer%d" % nh)
        # do the forward pass:
        PS = (PS_pre @ tl.weight.cpu().numpy().T
                   + tl.bias.cpu().numpy())
        PS = py_softmax(PS, 1)
        c, selflabels_ = optimize_L_sk(PS)
        _costs[nh] = c
        selflabels_in[nh] = selflabels_
        selflabels = selflabels_in
    return selflabels

## Training utils

In [12]:
def adjust_learning_rate(optimizer, epoch):
    """Sets the learning rate to the initial LR decayed by 10 every 30 epochs"""
    lr = alr
    if epochs == 200:
        if epoch >= 80:
            lr = alr * (0.1 ** ((epoch - 80) // 40))  # i.e. 120, 160
            print(lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    elif epochs == 400:
        if epoch >= 160:
            lr = alr * (0.1 ** ((epoch - 160) // 80))  # i.e. 240,320
            print(lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    elif epochs == 800:
        if epoch >= 320:
            lr = alr * (0.1 ** ((epoch - 320) // 160))  # i.e. 480, 640
            print(lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr
    elif epochs == 1600:
        if epoch >= 640:
            lr = alr * (0.1 ** ((epoch - 640) // 320))
            print(lr)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

In [13]:
def feature_return_switch(model, bool=True):
    """
    switch between network output or conv5features
        if True: changes switch s.t. forward pass returns post-conv5 features
        if False: changes switch s.t. forward will give full network output
    """
    if bool:
        model.headcount = 1
    else:
        model.headcount = hc
    model.return_feature = bool

In [14]:
def train(epoch, selflabels):
    print('\nEpoch: %d' % epoch)
    print(name)
    adjust_learning_rate(optimizer, epoch)
    train_loss = AverageMeter()
    data_time = AverageMeter()
    batch_time = AverageMeter()

    # switch to train mode
    model.train()

    end = time.time()
    
    for batch_idx, (inputs, targets, indexes) in enumerate(iterate_minibatches(X_train, y_train, batch_size, shuffle=True)):
        inputs = inputs.float().to(device)
        niter = epoch * N//batch_size + batch_idx
        if len(optimize_times) > 0 and niter * batch_size >= optimize_times[-1]:
            with torch.no_grad():
                _ = optimize_times.pop()
                if hc >1:
                    feature_return_switch(model, True)
                selflabels = opt_sk(model, selflabels, epoch)
                if hc >1:
                    feature_return_switch(model, False)
        data_time.update(time.time() - end)
        inputs, targets = inputs.to(device), targets.to(device)#, indexes.to(device)
        optimizer.zero_grad()

        outputs = model(inputs)
        if hc == 1:
            loss = criterion(outputs, selflabels[indexes])
        else:
            loss = torch.mean(torch.stack([criterion(outputs[h],
                                                     selflabels[h, indexes]) for h in range(hc)]))

        loss.backward()
        optimizer.step()

        train_loss.update(loss.item(), inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()
        if True:
#         if batch_idx % 10 == 0:
            print('Epoch: [{}][{}/{}]'
                  'Time: {batch_time.val:.3f} ({batch_time.avg:.3f}) '
                  'Data: {data_time.val:.3f} ({data_time.avg:.3f}) '
                  'Loss: {train_loss.val:.4f} ({train_loss.avg:.4f})'.format(
                epoch, batch_idx, N//batch_size, batch_time=batch_time, data_time=data_time, train_loss=train_loss))
#             writer.add_scalar("loss", loss.item(), batch_idx*512 +epoch*len(trainloader.dataset))
    return selflabels

## Model initialization

In [15]:
model = resnet18(num_classes=numc, in_channel=dims_num)

In [16]:
optimize_times = ((epochs + 1.0001)*N*(np.linspace(0, 1, nopts))[::-1]).tolist()
optimize_times = [(epochs +10)*N] + optimize_times
print('We will optimize L at epochs:', [np.round(1.0*t/N, 2) for t in optimize_times], flush=True)

We will optimize L at epochs: [410.0, 401.0, 400.0, 398.99, 397.99, 396.98, 395.98, 394.97, 393.97, 392.96, 391.95, 390.95, 389.94, 388.94, 387.93, 386.93, 385.92, 384.92, 383.91, 382.91, 381.9, 380.9, 379.89, 378.89, 377.88, 376.88, 375.87, 374.87, 373.86, 372.86, 371.85, 370.85, 369.84, 368.84, 367.83, 366.83, 365.82, 364.82, 363.81, 362.81, 361.8, 360.8, 359.79, 358.79, 357.78, 356.78, 355.77, 354.77, 353.76, 352.76, 351.75, 350.75, 349.74, 348.74, 347.73, 346.73, 345.72, 344.72, 343.71, 342.71, 341.7, 340.7, 339.69, 338.69, 337.68, 336.68, 335.67, 334.67, 333.66, 332.66, 331.65, 330.65, 329.64, 328.64, 327.63, 326.63, 325.62, 324.62, 323.61, 322.61, 321.6, 320.6, 319.59, 318.59, 317.58, 316.58, 315.57, 314.57, 313.56, 312.56, 311.55, 310.55, 309.54, 308.54, 307.53, 306.53, 305.52, 304.52, 303.51, 302.51, 301.5, 300.5, 299.49, 298.49, 297.48, 296.48, 295.47, 294.47, 293.46, 292.46, 291.45, 290.45, 289.44, 288.44, 287.43, 286.43, 285.42, 284.42, 283.41, 282.41, 281.4, 280.4, 279.39, 

In [17]:
optimize_times

[1008190,
 986059.2459,
 983587.9194691728,
 981116.5930383457,
 978645.2666075188,
 976173.9401766917,
 973702.6137458646,
 971231.2873150375,
 968759.9608842104,
 966288.6344533834,
 963817.3080225564,
 961345.9815917292,
 958874.6551609022,
 956403.3287300752,
 953932.002299248,
 951460.675868421,
 948989.349437594,
 946518.0230067669,
 944046.6965759398,
 941575.3701451126,
 939104.0437142856,
 936632.7172834587,
 934161.3908526315,
 931690.0644218044,
 929218.7379909774,
 926747.4115601503,
 924276.0851293232,
 921804.7586984962,
 919333.4322676691,
 916862.1058368421,
 914390.779406015,
 911919.4529751878,
 909448.1265443609,
 906976.8001135339,
 904505.4736827067,
 902034.1472518796,
 899562.8208210525,
 897091.4943902255,
 894620.1679593985,
 892148.8415285713,
 889677.5150977443,
 887206.1886669173,
 884734.8622360901,
 882263.5358052631,
 879792.2093744361,
 877320.882943609,
 874849.5565127819,
 872378.2300819547,
 869906.9036511277,
 867435.5772203008,
 864964.2507894736,
 

In [18]:
# init selflabels randomly
if hc == 1:
    selflabels = np.zeros(N, dtype=np.int32)
    for qq in range(N):
        selflabels[qq] = qq % ncl
    selflabels = np.random.permutation(selflabels)
    selflabels = torch.LongTensor(selflabels).to(device)
else:
    selflabels = np.zeros((hc, N), dtype=np.int32)
    for nh in range(hc):
        for _i in range(N):
            selflabels[nh, _i] = _i % numc[nh]
        selflabels[nh] = np.random.permutation(selflabels[nh])
    selflabels = torch.LongTensor(selflabels).to(device)

In [19]:
optimizer = optim.SGD(model.parameters(), lr=lr, momentum=momentum, weight_decay=5e-4)
model = model.to(device)
criterion = nn.CrossEntropyLoss()

In [20]:
name = "ResNet1D"
writer = SummaryWriter(f'./runs/ERing/{name}')

## Training! 
Takes a couple of minutes per epoch

In [21]:
def my_kNN(net, K, sigma=0.1, dim=128, use_pca=False):
    net.eval()
    # this part is ugly but made to be backwards-compatible. there was a change in cifar dataset's structure.
    trainLabels = y_train
    LEN = N
    C = trainLabels.max() + 1

    trainFeatures = torch.zeros((magic_dim, LEN))  # , device='cuda:0') # dim
    normalize = Normalize()
    for batch_idx, (inputs, targets, _) in enumerate(iterate_minibatches(X_train, y_train, batch_size, shuffle=False)):
        batchSize = batch_size
        inputs = inputs.cuda()
        features = net(inputs.float())
        if not use_pca:
            features = normalize(features)
        tmp = trainFeatures[:, batch_idx * batchSize:batch_idx * batchSize + batchSize]
        trainFeatures[:, batch_idx * batchSize:batch_idx * batchSize + batchSize] = features.data.t().cpu()
        
    if use_pca:
        comps = 128
        print('doing PCA with %s components'%comps, end=' ')
        from sklearn.decomposition import PCA
        pca = PCA(n_components=comps, whiten=False)
        trainFeatures = pca.fit_transform(trainFeatures.numpy().T)
        trainFeatures = torch.Tensor(trainFeatures)
        trainFeatures = normalize(trainFeatures).t()
        print('..done')
    def eval_k_s(K_,sigma_):
        total = 0
        top1 = 0.
        top5 = 0.

        with torch.no_grad():
            retrieval_one_hot = torch.zeros(K_, C)# .cuda()
            for batch_idx, (inputs, targets, _) in enumerate(iterate_minibatches(X_test, y_test, batch_size, shuffle=False)):
                targets = targets # .cuda(async=True) # or without async for py3.7
                inputs = inputs.cuda()
                batchSize = batch_size
                features = net(inputs)
                if use_pca:
                    features = pca.transform(features.cpu().numpy())
                    features = torch.Tensor(features).cuda()
                features = normalize(features).cpu()

                dist = torch.mm(features, trainFeatures)

                yd, yi = dist.topk(K_, dim=1, largest=True, sorted=True)
                candidates = trainLabels.view(1, -1).expand(batchSize, -1)
                retrieval = torch.gather(candidates, 1, yi).long()

                retrieval_one_hot.resize_(batchSize * K_, C).zero_()
                retrieval_one_hot.scatter_(1, retrieval.view(-1, 1), 1.)
                
                yd_transform = yd.clone().div_(sigma_).exp_()
                probs = torch.sum(torch.mul(retrieval_one_hot.view(batchSize, -1, C),
                                            yd_transform.view(batchSize, -1, 1)),
                                  1)
                _, predictions = probs.sort(1, True)

                # Find which predictions match the target
                correct = predictions.eq(targets.data.view(-1, 1))

                top1 = top1 + correct.narrow(1, 0, 1).sum().item()
                top5 = top5 + correct.narrow(1, 0, 5).sum().item()

                total += targets.size(0)

        print(f"{K_}-NN,s={sigma_}: TOP1: ", top1 * 100. / total)
        return top1 / total

    if isinstance(K, list):
        res = []
        for K_ in K:
            for sigma_ in sigma:
                res.append(eval_k_s(K_, sigma_))
        return res
    else:
        res = eval_k_s(K, sigma)
        return res

In [22]:
import time

start = time.time()
for epoch in range(start_epoch, start_epoch + epochs):
    print('Size pf grid: ', len(optimize_times))
    selflabels = train(epoch, selflabels)
    feature_return_switch(model, True)
    
    acc = my_kNN(model, K=10, sigma=0.1, dim=knn_dim)
    feature_return_switch(model, False)
#     writer.add_scalar("accuracy kNN", acc, epoch)
    if acc > best_acc:
        print('Saving..')
        state = {
            'net': model.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'opt': optimizer.state_dict(),
            'L': selflabels,
        }
        if not os.path.isdir(exp):
            os.mkdir(exp)
        torch.save(state, '%s/best_ckpt.t7' % (exp))
        best_acc = acc
    if epoch % 100 == 0:
        print('Saving..')
        state = {
            'net': model.state_dict(),
            'opt': optimizer.state_dict(),
            'acc': acc,
            'epoch': epoch,
            'L': selflabels,
        }
        if not os.path.isdir(exp):
            os.mkdir(exp)
        torch.save(state, '%s/ep%s.t7' % (exp, epoch))
    if epoch % 50 == 0:
        print("hello")
        feature_return_switch(model, True)
        acc = my_kNN(model, K=[50, 10], sigma=[0.1, 0.5], dim=knn_dim, use_pca=True)
        i = 0
#         for num_nn in [50, 10]:
#             for sig in [0.1, 0.5]:
#                 writer.add_scalar('knn%s-%s' % (num_nn, sig), acc[i], epoch)
#                 i += 1
        feature_return_switch(model, False)
    print('best accuracy: {:.2f}'.format(best_acc * 100))
end = time.time()

checkpoint = torch.load('%s'%exp+'/best_ckpt.t7' )
model.load_state_dict(checkpoint['net'])
feature_return_switch(model, True)
acc = my_kNN(model, K=10, sigma=0.1, dim=knn_dim, use_pca=True)

Size pf grid:  401

Epoch: 0
ResNet1D
error:  2.382538610845586e-13 step  11
cost:  4.579266110928804
opt took 0.00min,   11iters
Epoch: [0][0/24]Time: 0.301 (0.301) Data: 0.260 (0.260) Loss: 5.0348 (5.0348)
Epoch: [0][1/24]Time: 0.029 (0.165) Data: 0.001 (0.131) Loss: 4.9830 (5.0089)
Epoch: [0][2/24]Time: 0.028 (0.119) Data: 0.001 (0.088) Loss: 5.0335 (5.0171)
Epoch: [0][3/24]Time: 0.028 (0.097) Data: 0.001 (0.066) Loss: 4.9675 (5.0047)
Epoch: [0][4/24]Time: 0.025 (0.082) Data: 0.001 (0.053) Loss: 4.9766 (4.9991)
Epoch: [0][5/24]Time: 0.025 (0.073) Data: 0.001 (0.044) Loss: 4.9950 (4.9984)
Epoch: [0][6/24]Time: 0.024 (0.066) Data: 0.001 (0.038) Loss: 4.9466 (4.9910)
Epoch: [0][7/24]Time: 0.024 (0.061) Data: 0.001 (0.033) Loss: 4.9021 (4.9799)
Epoch: [0][8/24]Time: 0.024 (0.057) Data: 0.001 (0.030) Loss: 4.9059 (4.9717)
Epoch: [0][9/24]Time: 0.024 (0.053) Data: 0.001 (0.027) Loss: 4.8501 (4.9595)
Epoch: [0][10/24]Time: 0.023 (0.051) Data: 0.001 (0.025) Loss: 4.8659 (4.9510)
Epoch: [0][

10-NN,s=0.1: TOP1:  44.791666666666664
best accuracy: 46.42
Size pf grid:  397

Epoch: 4
ResNet1D
Epoch: [4][0/24]Time: 0.024 (0.024) Data: 0.001 (0.001) Loss: 3.8785 (3.8785)
error:  1.694200335577989e-13 step  11
cost:  4.21391828745979
opt took 0.00min,   11iters
Epoch: [4][1/24]Time: 0.254 (0.139) Data: 0.226 (0.114) Loss: 3.9255 (3.9020)
Epoch: [4][2/24]Time: 0.034 (0.104) Data: 0.001 (0.076) Loss: 3.7410 (3.8484)
Epoch: [4][3/24]Time: 0.049 (0.090) Data: 0.009 (0.059) Loss: 3.8194 (3.8411)
Epoch: [4][4/24]Time: 0.027 (0.078) Data: 0.005 (0.048) Loss: 3.7375 (3.8204)
Epoch: [4][5/24]Time: 0.024 (0.069) Data: 0.001 (0.040) Loss: 3.7057 (3.8013)
Epoch: [4][6/24]Time: 0.031 (0.063) Data: 0.001 (0.035) Loss: 3.6077 (3.7736)
Epoch: [4][7/24]Time: 0.035 (0.060) Data: 0.001 (0.031) Loss: 3.6430 (3.7573)
Epoch: [4][8/24]Time: 0.032 (0.057) Data: 0.001 (0.027) Loss: 3.6858 (3.7494)
Epoch: [4][9/24]Time: 0.030 (0.054) Data: 0.001 (0.025) Loss: 3.5045 (3.7249)
Epoch: [4][10/24]Time: 0.028 (0

10-NN,s=0.1: TOP1:  47.875
Saving..
best accuracy: 47.88
Size pf grid:  393

Epoch: 8
ResNet1D
Epoch: [8][0/24]Time: 0.021 (0.021) Data: 0.001 (0.001) Loss: 2.2580 (2.2580)
Epoch: [8][1/24]Time: 0.022 (0.022) Data: 0.001 (0.001) Loss: 2.3121 (2.2850)
error:  8.459899447643693e-14 step  11
cost:  3.3880880794283157
opt took 0.00min,   11iters
Epoch: [8][2/24]Time: 0.235 (0.093) Data: 0.210 (0.070) Loss: 2.3247 (2.2983)
Epoch: [8][3/24]Time: 0.028 (0.076) Data: 0.001 (0.053) Loss: 2.0861 (2.2452)
Epoch: [8][4/24]Time: 0.026 (0.066) Data: 0.001 (0.043) Loss: 2.2957 (2.2553)
Epoch: [8][5/24]Time: 0.026 (0.060) Data: 0.001 (0.036) Loss: 2.3754 (2.2753)
Epoch: [8][6/24]Time: 0.024 (0.055) Data: 0.001 (0.031) Loss: 2.2043 (2.2652)
Epoch: [8][7/24]Time: 0.021 (0.050) Data: 0.001 (0.027) Loss: 2.2647 (2.2651)
Epoch: [8][8/24]Time: 0.021 (0.047) Data: 0.001 (0.024) Loss: 2.4034 (2.2805)
Epoch: [8][9/24]Time: 0.021 (0.045) Data: 0.001 (0.022) Loss: 2.1089 (2.2633)
Epoch: [8][10/24]Time: 0.021 (0.

10-NN,s=0.1: TOP1:  48.791666666666664
best accuracy: 49.42
Size pf grid:  389

Epoch: 12
ResNet1D
Epoch: [12][0/24]Time: 0.022 (0.022) Data: 0.001 (0.001) Loss: 1.2621 (1.2621)
Epoch: [12][1/24]Time: 0.021 (0.022) Data: 0.001 (0.001) Loss: 1.2395 (1.2508)
error:  1.3788969965844444e-13 step  11
cost:  1.9401732355744015
opt took 0.00min,   11iters
Epoch: [12][2/24]Time: 0.349 (0.131) Data: 0.313 (0.105) Loss: 1.5926 (1.3647)
Epoch: [12][3/24]Time: 0.033 (0.106) Data: 0.001 (0.079) Loss: 1.1606 (1.3137)
Epoch: [12][4/24]Time: 0.031 (0.091) Data: 0.001 (0.063) Loss: 1.2141 (1.2938)
Epoch: [12][5/24]Time: 0.027 (0.081) Data: 0.001 (0.053) Loss: 1.4505 (1.3199)
Epoch: [12][6/24]Time: 0.024 (0.072) Data: 0.001 (0.046) Loss: 1.1665 (1.2980)
Epoch: [12][7/24]Time: 0.023 (0.066) Data: 0.001 (0.040) Loss: 1.3194 (1.3007)
Epoch: [12][8/24]Time: 0.023 (0.062) Data: 0.001 (0.036) Loss: 1.2158 (1.2912)
Epoch: [12][9/24]Time: 0.024 (0.058) Data: 0.001 (0.032) Loss: 1.2296 (1.2851)
Epoch: [12][10/24

Epoch: [15][22/24]Time: 0.022 (0.033) Data: 0.001 (0.011) Loss: 1.0695 (0.9769)
Epoch: [15][23/24]Time: 0.022 (0.032) Data: 0.001 (0.010) Loss: 0.7966 (0.9694)
10-NN,s=0.1: TOP1:  52.166666666666664
Saving..
best accuracy: 52.17
Size pf grid:  385

Epoch: 16
ResNet1D
Epoch: [16][0/24]Time: 0.024 (0.024) Data: 0.001 (0.001) Loss: 1.0427 (1.0427)
Epoch: [16][1/24]Time: 0.022 (0.023) Data: 0.001 (0.001) Loss: 0.8022 (0.9224)
Epoch: [16][2/24]Time: 0.022 (0.023) Data: 0.001 (0.001) Loss: 0.6536 (0.8328)
error:  4.9182879990894435e-14 step  11
cost:  1.2445062237870486
opt took 0.00min,   11iters
Epoch: [16][3/24]Time: 0.246 (0.079) Data: 0.219 (0.055) Loss: 0.9035 (0.8505)
Epoch: [16][4/24]Time: 0.027 (0.068) Data: 0.001 (0.045) Loss: 1.0572 (0.8918)
Epoch: [16][5/24]Time: 0.027 (0.061) Data: 0.001 (0.037) Loss: 0.8023 (0.8769)
Epoch: [16][6/24]Time: 0.027 (0.057) Data: 0.001 (0.032) Loss: 0.7594 (0.8601)
Epoch: [16][7/24]Time: 0.024 (0.052) Data: 0.001 (0.028) Loss: 0.8151 (0.8545)
Epoch:

...

Epoch: [382][7/24]Time: 0.022 (0.051) Data: 0.001 (0.027) Loss: 0.3175 (0.3168)
Epoch: [382][8/24]Time: 0.023 (0.048) Data: 0.001 (0.024) Loss: 0.3683 (0.3225)
Epoch: [382][9/24]Time: 0.023 (0.045) Data: 0.001 (0.022) Loss: 0.2457 (0.3149)
Epoch: [382][10/24]Time: 0.022 (0.043) Data: 0.001 (0.020) Loss: 0.3050 (0.3140)
Epoch: [382][11/24]Time: 0.022 (0.042) Data: 0.001 (0.019) Loss: 0.3684 (0.3185)
Epoch: [382][12/24]Time: 0.022 (0.040) Data: 0.001 (0.017) Loss: 0.3545 (0.3213)
Epoch: [382][13/24]Time: 0.022 (0.039) Data: 0.001 (0.016) Loss: 0.2654 (0.3173)
Epoch: [382][14/24]Time: 0.023 (0.038) Data: 0.001 (0.015) Loss: 0.2802 (0.3148)
Epoch: [382][15/24]Time: 0.022 (0.037) Data: 0.001 (0.014) Loss: 0.2614 (0.3115)
Epoch: [382][16/24]Time: 0.022 (0.036) Data: 0.001 (0.013) Loss: 0.2626 (0.3086)
Epoch: [382][17/24]Time: 0.022 (0.035) Data: 0.001 (0.013) Loss: 0.2249 (0.3039)
Epoch: [382][18/24]Time: 0.036 (0.035) Data: 0.001 (0.012) Loss: 0.2482 (0.3010)
Epoch: [382][19/24]Time: 0.030 

Epoch: [386][8/24]Time: 0.023 (0.023) Data: 0.001 (0.001) Loss: 0.2998 (0.3154)
Epoch: [386][9/24]Time: 0.023 (0.023) Data: 0.001 (0.001) Loss: 0.2506 (0.3090)
Epoch: [386][10/24]Time: 0.023 (0.023) Data: 0.001 (0.001) Loss: 0.2232 (0.3012)
Epoch: [386][11/24]Time: 0.023 (0.023) Data: 0.001 (0.001) Loss: 0.3035 (0.3014)
Epoch: [386][12/24]Time: 0.023 (0.023) Data: 0.001 (0.001) Loss: 0.2136 (0.2946)
Epoch: [386][13/24]Time: 0.023 (0.023) Data: 0.001 (0.001) Loss: 0.2921 (0.2944)
Epoch: [386][14/24]Time: 0.023 (0.023) Data: 0.001 (0.001) Loss: 0.3072 (0.2953)
Epoch: [386][15/24]Time: 0.023 (0.023) Data: 0.001 (0.001) Loss: 0.2939 (0.2952)
Epoch: [386][16/24]Time: 0.023 (0.023) Data: 0.001 (0.001) Loss: 0.2940 (0.2951)
Epoch: [386][17/24]Time: 0.023 (0.023) Data: 0.001 (0.001) Loss: 0.2901 (0.2948)
Epoch: [386][18/24]Time: 0.023 (0.023) Data: 0.001 (0.001) Loss: 0.1773 (0.2887)
Epoch: [386][19/24]Time: 0.022 (0.023) Data: 0.001 (0.001) Loss: 0.2273 (0.2856)
Epoch: [386][20/24]Time: 0.022

Epoch: [390][7/24]Time: 0.023 (0.053) Data: 0.001 (0.029) Loss: 0.2724 (0.2985)
Epoch: [390][8/24]Time: 0.023 (0.049) Data: 0.001 (0.026) Loss: 0.2712 (0.2955)
Epoch: [390][9/24]Time: 0.023 (0.047) Data: 0.001 (0.023) Loss: 0.2518 (0.2911)
Epoch: [390][10/24]Time: 0.022 (0.045) Data: 0.001 (0.021) Loss: 0.2989 (0.2918)
Epoch: [390][11/24]Time: 0.022 (0.043) Data: 0.001 (0.020) Loss: 0.3011 (0.2926)
Epoch: [390][12/24]Time: 0.022 (0.041) Data: 0.001 (0.018) Loss: 0.2976 (0.2930)
Epoch: [390][13/24]Time: 0.023 (0.040) Data: 0.001 (0.017) Loss: 0.2764 (0.2918)
Epoch: [390][14/24]Time: 0.023 (0.039) Data: 0.001 (0.016) Loss: 0.2947 (0.2920)
Epoch: [390][15/24]Time: 0.022 (0.038) Data: 0.001 (0.015) Loss: 0.2104 (0.2869)
Epoch: [390][16/24]Time: 0.022 (0.037) Data: 0.001 (0.014) Loss: 0.2877 (0.2869)
Epoch: [390][17/24]Time: 0.022 (0.036) Data: 0.001 (0.013) Loss: 0.2101 (0.2827)
Epoch: [390][18/24]Time: 0.022 (0.035) Data: 0.001 (0.013) Loss: 0.3005 (0.2836)
Epoch: [390][19/24]Time: 0.022 

Epoch: [394][6/24]Time: 0.023 (0.062) Data: 0.001 (0.035) Loss: 0.2697 (0.2883)
Epoch: [394][7/24]Time: 0.023 (0.057) Data: 0.001 (0.030) Loss: 0.2262 (0.2805)
Epoch: [394][8/24]Time: 0.023 (0.053) Data: 0.001 (0.027) Loss: 0.2600 (0.2782)
Epoch: [394][9/24]Time: 0.024 (0.050) Data: 0.001 (0.024) Loss: 0.2756 (0.2780)
Epoch: [394][10/24]Time: 0.023 (0.048) Data: 0.001 (0.022) Loss: 0.2594 (0.2763)
Epoch: [394][11/24]Time: 0.023 (0.046) Data: 0.001 (0.021) Loss: 0.2506 (0.2742)
Epoch: [394][12/24]Time: 0.023 (0.044) Data: 0.001 (0.019) Loss: 0.2678 (0.2737)
Epoch: [394][13/24]Time: 0.023 (0.043) Data: 0.001 (0.018) Loss: 0.2076 (0.2689)
Epoch: [394][14/24]Time: 0.023 (0.041) Data: 0.001 (0.017) Loss: 0.2646 (0.2687)
Epoch: [394][15/24]Time: 0.023 (0.040) Data: 0.001 (0.016) Loss: 0.2668 (0.2685)
Epoch: [394][16/24]Time: 0.023 (0.039) Data: 0.001 (0.015) Loss: 0.4003 (0.2763)
Epoch: [394][17/24]Time: 0.023 (0.038) Data: 0.001 (0.014) Loss: 0.3228 (0.2789)
Epoch: [394][18/24]Time: 0.023 (

Epoch: [398][1/24]Time: 0.298 (0.160) Data: 0.270 (0.136) Loss: 0.2568 (0.2400)
Epoch: [398][2/24]Time: 0.026 (0.115) Data: 0.001 (0.091) Loss: 0.3161 (0.2654)
Epoch: [398][3/24]Time: 0.028 (0.094) Data: 0.001 (0.069) Loss: 0.3068 (0.2758)
Epoch: [398][4/24]Time: 0.047 (0.084) Data: 0.005 (0.056) Loss: 0.2641 (0.2734)
Epoch: [398][5/24]Time: 0.021 (0.074) Data: 0.001 (0.047) Loss: 0.3460 (0.2855)
Epoch: [398][6/24]Time: 0.021 (0.066) Data: 0.001 (0.040) Loss: 0.2210 (0.2763)
Epoch: [398][7/24]Time: 0.021 (0.061) Data: 0.001 (0.035) Loss: 0.3795 (0.2892)
Epoch: [398][8/24]Time: 0.022 (0.056) Data: 0.001 (0.031) Loss: 0.3338 (0.2942)
Epoch: [398][9/24]Time: 0.021 (0.053) Data: 0.001 (0.028) Loss: 0.3093 (0.2957)
Epoch: [398][10/24]Time: 0.021 (0.050) Data: 0.001 (0.026) Loss: 0.2509 (0.2916)
Epoch: [398][11/24]Time: 0.021 (0.048) Data: 0.001 (0.024) Loss: 0.3715 (0.2983)
Epoch: [398][12/24]Time: 0.021 (0.046) Data: 0.001 (0.022) Loss: 0.2357 (0.2935)
Epoch: [398][13/24]Time: 0.021 (0.044

In [23]:
print (end-start)

621.6017904281616
